In [99]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.init import normal_


def get_config():
    mean = 0
    stddev = 0.1
    return mean, stddev


mean, stddev = get_config()


def Normal(tensor, mean=mean, stddev=stddev):
    "Re initialize the tensor with normal weights and custom mean and stddev"
    normal_(tensor, mean=mean, std=stddev)
    return None


class vBPR(nn.Module):
    """
    Creates a vBPR module, which learns the latent factors over
    the user and item interactions.

    For more details refer to the paper: https://arxiv.org/pdf/1510.01784.pdf
    """

    def __init__(self,
                 num_latent_factors,
                 num_visual_factors,
                 num_embedding_factors,
                 num_users,
                 num_items,
                 visual_features,
                 dropout=0.1):
        "Creates the weights matrices for storing factors"
        super(vBPR, self).__init__()
        self.K = num_latent_factors
        self.D = num_visual_factors
        self.F = num_embedding_factors

        self.n_u = num_users
        self.n_i = num_items

        mean, stddev = get_config()
        # declare latent factor matrices for users and items
        self.U_latent_factors = nn.Parameter(torch.randn(self.n_u, self.K))
        self.I_latent_factors = nn.Parameter(torch.randn(self.n_i, self.K))
        Normal(self.U_latent_factors)
        Normal(self.I_latent_factors)

        # declare visual factor matrices for users
        self.U_visual_factors = nn.Parameter(torch.randn(self.n_u, self.D))
        Normal(self.U_visual_factors)

        # embedding linear layer for projecting embedding to visual factors
        self.embedding_projection = nn.Linear(self.F, self.D)
        Normal(self.embedding_projection.weight)
        Normal(self.embedding_projection.bias)

        self.dropout = nn.Dropout(dropout)

        # visual bias
        self.beta_dash = nn.Parameter(torch.randn(1, self.F))
        Normal(self.beta_dash)

        # user bias and item bias
        self.user_bias = nn.Parameter(torch.zeros(self.n_u))
        self.item_bias = nn.Parameter(torch.zeros(self.n_i))
        Normal(self.user_bias)
        Normal(self.item_bias)

        self.visual_features = visual_features

        # TODO: include regularization

    def get_xui(self, u_s, i_s):
        "Get x_ui value for a bunch of user indices and item indices"
        I_visual_factors = self.dropout(
            self.embedding_projection(
                self.visual_features[i_s]))

        return self.user_bias[u_s] + self.item_bias[i_s] + \
            torch.bmm(
                self.U_latent_factors[u_s].unsqueeze(1),
                self.I_latent_factors[i_s].unsqueeze(2)
        ).squeeze() + \
            torch.bmm(
                self.U_visual_factors[u_s].unsqueeze(1),
                I_visual_factors.unsqueeze(2)
        ).squeeze() + \
            self.beta_dash.mm(
                self.visual_features[i_s].transpose(0, 1)
        ).squeeze()

    def forward(self, trg_batch):
        """Calculate the preferences of user, i, j pairs.

            Args:
                trg_batch: [batch, 3]
            Returns:
                A Tensor of shape [batch, 1]
        """
        user_indices = trg_batch[:, 0]
        i_indices = trg_batch[:, 1]
        j_indices = trg_batch[:, 2]
        return self.get_xui(user_indices, i_indices) - \
            self.get_xui(user_indices, j_indices)

def data_gen(train_data, batch_size=8):
    """
    Yields batches of training data with given batch size

    Args:
        train_data : Interaction, which contains user, i, count tuples

    Yields :
        batch of (u, i, j) tuples
    """
    interactions_dict = train_data.get_interaction_dict()
    num_items = train_data.num_items
    num_users = train_data.num_users

    for user in interactions_dict:
        for item in interactions_dict[user]:
            X = np.zeros((batch_size, 3))
            X[:, 0] = user
            X[:, 1] = item
            js = [np.random.randint(num_items) for _ in range(2 * batch_size)]
            row_index = 0
            for j in js:
                if j not in interactions_dict[user] and row_index < batch_size:
                    X[row_index, 2] = j
                    row_index += 1

            # just repeat the first row remaining times
            if row_index < batch_size:
                X[row_index, 2] = X[0, 2]
                row_index += 1

            yield torch.as_tensor(X, dtype=torch.long)


num_epochs = 1

In [3]:
import json
with open('data/subreddit_embeddings/subreddit_32', 'r') as f:
    subreddit_factors = json.loads(f.read())

In [4]:
subreddits_read = np.array([item[0] for item in subreddit_factors])

In [5]:
visual_features = np.array([item[1] for item in subreddit_factors]).astype(np.float32)

In [6]:
visual_features = visual_features / np.linalg.norm(visual_features, axis=1).reshape((-1, 1))

In [8]:
import csv

In [9]:
data = []
with open('interactions_5') as csvfile:
    datareader = csv.reader(csvfile, delimiter=' ')
    for subreddit, user, comments in datareader:
        data.append([user, subreddit, int(comments)])

In [10]:
from copy import deepcopy
intr = deepcopy(data)

In [11]:
import pandas as pd

In [12]:
data = pd.DataFrame.from_records(data)
data.columns = ['user', 'subreddit', 'comments']

data['user'] = data['user'].astype("category")
data['subreddit'] = data['subreddit'].astype("category")

In [13]:
from scipy.sparse import coo_matrix

In [14]:
# create a sparse matrix of all the artist/user/play triples
comments = coo_matrix((data['comments'].astype(float), 
                   (data['subreddit'].cat.codes, 
                    data['user'].cat.codes)))

In [15]:
subreddits = data['subreddit'].cat.categories.array.to_numpy()
users = data['user'].cat.categories.array.to_numpy()

In [16]:
print('Number of users for BPR model: %s' % len(users))
print('Number of subreddits for BPR model: %s' % len(subreddits))

Number of users for BPR model: 285103
Number of subreddits for BPR model: 13500


In [17]:
from utils import *

In [18]:
subreddits_index = item_to_index(subreddits)
users_index = item_to_index(users)

In [19]:
def train_test_split(coo_comments):
    """
    Omits random user subreddit interactions, zeros them out 
    and appends them to the test list.
    """
    csr_comments = coo_comments.tocsr()
    
    data = defaultdict(lambda: [])
    with open('interactions_5') as csvfile:
        datareader = csv.reader(csvfile, delimiter=' ')        
        for subreddit, user, comments in tqdm.tqdm_notebook(datareader):
            data[user].append((subreddit, comments))
    
    train_set = []
    test_set = []
    
    for user, items in tqdm.tqdm_notebook(data.items()):
        np.random.shuffle(items)
        test_item = items[0]
        test_comments = items[1]
        
        test_subreddit = test_item[0]
        # zero out a user item interaction
        csr_comments[subreddits_index[test_subreddit], users_index[user]] = 0
        
        test_set.append([test_subreddit, user, int(comments)])
        
        for item in items[1:]:
            train_set.append([item[0], user, int(item[1])])
        
    csr_comments.eliminate_zeros()
    return train_set, test_set, csr_comments.tocoo()

In [20]:
from collections import defaultdict
import tqdm
from tqdm import tqdm_notebook
train_set, test_set, comments = train_test_split(comments)

In [21]:
len(train_set)

801933

In [22]:
len(test_set)

285103

In [23]:
subreddits_read

array(['exmormon', 'CanadaPolitics', 'AdviceAnimals', ..., 'SCNewsHelper',
       'infinitejukebox', 'supremecommander'], dtype='<U21')

In [24]:
len(subreddits_read)

14842

In [25]:
subreddits_read_dict = {item[0]: np.array(item[1]).astype(np.float32) for item in subreddit_factors}

In [30]:
subreddits_read_dict['100DaysofKeto']

array([-3.1117148 ,  0.0611029 , -1.1465515 , -0.58117855,  2.117647  ,
       -2.6484606 , -0.08750109, -4.2095976 ,  0.5333132 ,  1.2737285 ,
        0.5197254 ,  1.759372  ,  0.907367  ,  2.7493865 , -1.2573737 ,
       -1.2567716 , -2.6386232 , -1.2101593 ,  4.0103564 , -0.31075937,
        1.4910036 , -1.4997488 ,  0.9246351 , -2.0805774 ,  0.7282468 ,
       -1.4214854 ,  2.0188582 , -2.818061  ,  0.60485935, -3.3702948 ,
       -0.11576921,  0.02674413], dtype=float32)

In [28]:
visual_f = np.array([subreddits_read_dict[sr] for sr in subreddits])

In [31]:
visual_features = visual_f / np.linalg.norm(visual_f, axis=1).reshape((-1, 1))

In [32]:
visual_features

array([[-0.2865164 ,  0.00562615, -0.10557067, ..., -0.3103256 ,
        -0.01065965,  0.00246251],
       [-0.07309087,  0.24505946,  0.02452864, ..., -0.38007447,
        -0.00202844, -0.1574662 ],
       [-0.23416445,  0.15118076, -0.27649128, ..., -0.38081914,
        -0.1973587 ,  0.02927169],
       ...,
       [-0.25510266,  0.06433506, -0.13156392, ..., -0.28342536,
        -0.38534567, -0.1107089 ],
       [ 0.21625948,  0.18403211,  0.17586964, ..., -0.160632  ,
        -0.2234102 , -0.22328632],
       [-0.20562035, -0.03368286, -0.06545708, ..., -0.00839527,
        -0.21790776, -0.1680204 ]], dtype=float32)

In [36]:
comments

<13500x285103 sparse matrix of type '<class 'numpy.float64'>'
	with 801933 stored elements in COOrdinate format>

In [34]:
train_set_ = [[users_index[user], subreddits_index[sr], count] for sr, user, count in train_set]

In [37]:
interact = Interaction(interactions=train_set_, num_items=13500, num_users=285103)

In [78]:
gen = data_gen(interact, 1)

In [65]:
torch.log(torch.Tensor([10]))

tensor([2.3026])

In [80]:
nb = 50000
i = 0
for batch in gen:
    if i >= nb:
        break
    print(
        batch)
    i += 1

tensor([[136512,   2338,  10169]])
tensor([[136512,   8920,   1171]])
tensor([[136512,   9233,  11361]])
tensor([[136512,   8457,   5223]])
tensor([[89351,  8920,  3101]])
tensor([[89351,  7403,  1008]])
tensor([[89351, 11046,   596]])
tensor([[89351,  4940,   812]])
tensor([[262752,   8920,  13254]])
tensor([[262752,   8917,  10759]])
tensor([[262752,  12782,   6877]])
tensor([[262752,    541,   9402]])
tensor([[220685,  10705,    167]])
tensor([[220685,   8920,   2574]])
tensor([[220685,  10179,  10578]])
tensor([[228830,   8920,   6259]])
tensor([[127385,  12867,    910]])
tensor([[183698,   1039,  10886]])
tensor([[203629,   6295,   9145]])
tensor([[203629,  10888,   1119]])
tensor([[203629,  10980,   4832]])
tensor([[203629,   8920,   6604]])
tensor([[203629,  10707,   8183]])
tensor([[203629,   9233,   4960]])
tensor([[232075,  13382,    667]])
tensor([[232075,   8920,   1550]])
tensor([[11761,  8920,  4052]])
tensor([[234864,   7403,   1015]])
tensor([[187335,  12604,   5735]])


tensor([[71809,  5393,  5269]])
tensor([[71809,  9233,  4316]])
tensor([[71809, 13317, 12064]])
tensor([[71809,  9301,  2921]])
tensor([[267288,   3449,   9889]])
tensor([[267288,   8920,   6864]])
tensor([[225474,  11602,   9828]])
tensor([[225474,    541,  11547]])
tensor([[225474,   6511,   9909]])
tensor([[225474,   7403,  12661]])
tensor([[225474,    499,   9798]])
tensor([[225474,   8929,   2361]])
tensor([[225474,   8064,  10654]])
tensor([[225474,  10045,  13359]])
tensor([[12867,  8829,  3876]])
tensor([[12867,  8920,   214]])
tensor([[12867,  8929, 12261]])
tensor([[12867,   248, 11882]])
tensor([[12867,  2582,  2118]])
tensor([[12867, 13382,  1445]])
tensor([[12867, 10728,   677]])
tensor([[12867,  3921,  3992]])
tensor([[12867, 12641,  3842]])
tensor([[12867, 11952,  9778]])
tensor([[12867, 12923,   773]])
tensor([[12867, 10888,  2749]])
tensor([[69417,  8920,   151]])
tensor([[69417, 10728, 13459]])
tensor([[69417,  6204,  3155]])
tensor([[167602,  10179,   9096]])
tensor(

tensor([[11502, 13011,  7292]])
tensor([[11502,  9233,  2813]])
tensor([[11502,  8920,  4523]])
tensor([[11502, 11370,  4832]])
tensor([[237173,   2790,   7647]])
tensor([[237173,   7466,   8389]])
tensor([[105941,  12879,   6756]])
tensor([[105941,   8920,  10161]])
tensor([[227849,    541,   9186]])
tensor([[227849,    558,   4267]])
tensor([[227849,   5433,   4070]])
tensor([[227849,   8920,   9964]])
tensor([[146909,   8920,    805]])
tensor([[197556,  10196,  11056]])
tensor([[269581,   8919,   3330]])
tensor([[269581,   8920,   7915]])
tensor([[169992,   6494,  13164]])
tensor([[169992,  13382,   8062]])
tensor([[169992,   8920,   5338]])
tensor([[259701,   3542,   5603]])
tensor([[259701,   4930,  13334]])
tensor([[259701,  10182,   2892]])
tensor([[259701,   7502,   9546]])
tensor([[213351,   8920,   6556]])
tensor([[108130,   8920,   1764]])
tensor([[99424,   541,  8284]])
tensor([[99424,  6642, 10649]])
tensor([[99424,  6763, 13263]])
tensor([[171330,  13329,   7799]])
tensor

In [81]:
len(train_set)

801933

In [184]:
def train(num_latent_factors,
        num_visual_factors,
        num_embedding_factors,
        num_users,
        num_items,
        train_data_gen,
        visual_features,
        dropout=0.1,
        num_batches=1000,
        learning_rate=0.01):
    "trains the network over the training data"
    model = vBPR(num_latent_factors=num_latent_factors,
                 num_visual_factors=num_visual_factors,
                 num_embedding_factors=num_embedding_factors,
                 num_users=num_users,
                 num_items=num_items,
                 visual_features=visual_features,
                 dropout=dropout)

    # initialize variables
    loss = 0
    print_losses = []

    # can try other optimizers here
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

    num_batches = num_batches
    i = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        for trg_batch in train_data_gen:
            if i >= num_batches:
                break

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimizer
            outputs = model(trg_batch)
            #print(outputs)
            loss = BPRLoss(outputs)
            #print(loss.item())
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            if i % 100 == 99:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

            i += 1

    print('Finished Training')
    return model

In [103]:
def BPRLoss(batch_xuij):
    "Return the loss for a batch of xuij predictions"
    loss = (1.0 - torch.sigmoid(batch_xuij))
    return loss.sum()

    # return -torch.log(torch.sigmoid(batch_xuij)).sum()

In [186]:
gen = data_gen(interact, 2)
model = train(
        num_latent_factors = 16,
        num_visual_factors = 16,
        num_embedding_factors = 32,
        num_users = interact.num_users,
        num_items = interact.num_items,
        train_data_gen = gen,
        visual_features = torch.as_tensor(visual_features),
        dropout=0.1,
        num_batches=5000,
        learning_rate=0.1
)

[1,   100] loss: 0.493
[1,   200] loss: 0.410
[1,   300] loss: 0.298
[1,   400] loss: 0.204
[1,   500] loss: 0.241
[1,   600] loss: 0.249
[1,   700] loss: 0.217
[1,   800] loss: 0.269
[1,   900] loss: 0.327
[1,  1000] loss: 0.311
[1,  1100] loss: 0.242
[1,  1200] loss: 0.296
[1,  1300] loss: 0.276
[1,  1400] loss: 0.263
[1,  1500] loss: 0.220
[1,  1600] loss: 0.254
[1,  1700] loss: 0.246
[1,  1800] loss: 0.222
[1,  1900] loss: 0.227
[1,  2000] loss: 0.257
[1,  2100] loss: 0.257
[1,  2200] loss: 0.216
[1,  2300] loss: 0.285
[1,  2400] loss: 0.331
[1,  2500] loss: 0.174
[1,  2600] loss: 0.105
[1,  2700] loss: 0.268
[1,  2800] loss: 0.157
[1,  2900] loss: 0.233
[1,  3000] loss: 0.255
[1,  3100] loss: 0.134
[1,  3200] loss: 0.304
[1,  3300] loss: 0.267
[1,  3400] loss: 0.167
[1,  3500] loss: 0.236
[1,  3600] loss: 0.200
[1,  3700] loss: 0.179
[1,  3800] loss: 0.191
[1,  3900] loss: 0.170
[1,  4000] loss: 0.238
[1,  4100] loss: 0.165
[1,  4200] loss: 0.130
[1,  4300] loss: 0.140
[1,  4400] 

In [187]:
item_f = model.I_latent_factors.detach().numpy()

In [188]:
def dot_(x, y):
    return item_f[subreddits_index[x]].dot(item_f[subreddits_index[y]])

In [189]:
dot_('OnePiece', 'manga')

-0.023544954

In [190]:
def auc(test_set, model, num_subreddits):
    """
    Returns the auc score on a test data set
    """
    num_users = len(test_set)
    total = 0

    
    # treat the signal as 1 as per the implicit bpr paper
    for subreddit, user, signal in tqdm.tqdm_notebook(test_set):
        u = users_index[user]
        i = subreddits_index[subreddit]

        # x_ui = user_factors[u].dot(subreddit_factors[i])

        js = []

        for j in range(0, num_subreddits):
            if j != i and j not in E_u[u]:
                js.append(j)
                
        batch = torch.LongTensor(len(js), 3)
        for k in range(len(js)):
            batch[k][0] = u
            batch[k][1] = i
            batch[k][2] = js[k]
        
        # print(batch)
        # print(model(batch))
        out = model(batch)
        out = out.detach().numpy()
        total += np.sum(np.heaviside(out, 0)) / len(js)

        # total += np.sum(np.heaviside(x_ui - \
        #                user_factors[u].dot(subreddit_factors[js].T), 0)) / len(js)

    return  total / num_users

In [110]:
# create E(u) list for each user and store it use ids instead of names to store them
E_u = defaultdict(lambda : set())

for subreddit, user, _ in tqdm.tqdm_notebook(train_set):
        E_u[users_index[user]].add(subreddits_index[subreddit])
            
for subreddit, user, _ in tqdm.tqdm_notebook(test_set):
        E_u[users_index[user]].add(subreddits_index[subreddit])

In [191]:
auc(test_set[:100], model, num_subreddits=interact.num_items)

0.9031613547393769

In [204]:
aucs = []

for latent_factor in [8, 16, 32, 64, 128]:
    gen = data_gen(interact, 2)
    model = train(
            num_latent_factors = latent_factor,
            num_visual_factors = latent_factor,
            num_embedding_factors = 32,
            num_users = interact.num_users,
            num_items = interact.num_items,
            train_data_gen = gen,
            visual_features = torch.as_tensor(visual_features),
            dropout=0.1,
            num_batches=5000,
            learning_rate=0.1
    )
    
    auc_ = auc(test_set[:100], model, num_subreddits=interact.num_items)
    print("Latent factors %s: Auc: %s:" %(latent_factor, auc_))
    aucs.append(auc_)

[1,   100] loss: 0.507
[1,   200] loss: 0.425
[1,   300] loss: 0.285
[1,   400] loss: 0.196
[1,   500] loss: 0.272
[1,   600] loss: 0.242
[1,   700] loss: 0.259
[1,   800] loss: 0.246
[1,   900] loss: 0.388
[1,  1000] loss: 0.329
[1,  1100] loss: 0.272
[1,  1200] loss: 0.364
[1,  1300] loss: 0.252
[1,  1400] loss: 0.258
[1,  1500] loss: 0.230
[1,  1600] loss: 0.281
[1,  1700] loss: 0.215
[1,  1800] loss: 0.295
[1,  1900] loss: 0.194
[1,  2000] loss: 0.272
[1,  2100] loss: 0.286
[1,  2200] loss: 0.265
[1,  2300] loss: 0.326
[1,  2400] loss: 0.267
[1,  2500] loss: 0.128
[1,  2600] loss: 0.114
[1,  2700] loss: 0.204
[1,  2800] loss: 0.143
[1,  2900] loss: 0.159
[1,  3000] loss: 0.250
[1,  3100] loss: 0.167
[1,  3200] loss: 0.331
[1,  3300] loss: 0.299
[1,  3400] loss: 0.189
[1,  3500] loss: 0.243
[1,  3600] loss: 0.141
[1,  3700] loss: 0.208
[1,  3800] loss: 0.184
[1,  3900] loss: 0.219
[1,  4000] loss: 0.272
[1,  4100] loss: 0.140
[1,  4200] loss: 0.094
[1,  4300] loss: 0.141
[1,  4400] 

Latent factors 8: Auc: 0.9005884623283035:
[1,   100] loss: 0.501
[1,   200] loss: 0.396
[1,   300] loss: 0.310
[1,   400] loss: 0.223
[1,   500] loss: 0.210
[1,   600] loss: 0.210
[1,   700] loss: 0.250
[1,   800] loss: 0.244
[1,   900] loss: 0.325
[1,  1000] loss: 0.356
[1,  1100] loss: 0.286
[1,  1200] loss: 0.290
[1,  1300] loss: 0.250
[1,  1400] loss: 0.267
[1,  1500] loss: 0.241
[1,  1600] loss: 0.253
[1,  1700] loss: 0.217
[1,  1800] loss: 0.219
[1,  1900] loss: 0.164
[1,  2000] loss: 0.249
[1,  2100] loss: 0.262
[1,  2200] loss: 0.236
[1,  2300] loss: 0.266
[1,  2400] loss: 0.292
[1,  2500] loss: 0.146
[1,  2600] loss: 0.142
[1,  2700] loss: 0.205
[1,  2800] loss: 0.171
[1,  2900] loss: 0.249
[1,  3000] loss: 0.264
[1,  3100] loss: 0.168
[1,  3200] loss: 0.331
[1,  3300] loss: 0.261
[1,  3400] loss: 0.175
[1,  3500] loss: 0.208
[1,  3600] loss: 0.187
[1,  3700] loss: 0.196
[1,  3800] loss: 0.220
[1,  3900] loss: 0.201
[1,  4000] loss: 0.236
[1,  4100] loss: 0.156
[1,  4200] los

Latent factors 16: Auc: 0.9000449631079535:
[1,   100] loss: 0.491
[1,   200] loss: 0.391
[1,   300] loss: 0.258
[1,   400] loss: 0.231
[1,   500] loss: 0.293
[1,   600] loss: 0.265
[1,   700] loss: 0.228
[1,   800] loss: 0.265
[1,   900] loss: 0.324
[1,  1000] loss: 0.316
[1,  1100] loss: 0.311
[1,  1200] loss: 0.314
[1,  1300] loss: 0.227
[1,  1400] loss: 0.368
[1,  1500] loss: 0.266
[1,  1600] loss: 0.241
[1,  1700] loss: 0.143
[1,  1800] loss: 0.203
[1,  1900] loss: 0.237
[1,  2000] loss: 0.182
[1,  2100] loss: 0.290
[1,  2200] loss: 0.191
[1,  2300] loss: 0.284
[1,  2400] loss: 0.337
[1,  2500] loss: 0.160
[1,  2600] loss: 0.117
[1,  2700] loss: 0.201
[1,  2800] loss: 0.205
[1,  2900] loss: 0.250
[1,  3000] loss: 0.260
[1,  3100] loss: 0.141
[1,  3200] loss: 0.276
[1,  3300] loss: 0.260
[1,  3400] loss: 0.193
[1,  3500] loss: 0.177
[1,  3600] loss: 0.195
[1,  3700] loss: 0.200
[1,  3800] loss: 0.219
[1,  3900] loss: 0.213
[1,  4000] loss: 0.286
[1,  4100] loss: 0.175
[1,  4200] lo

Latent factors 32: Auc: 0.8960811226825817:
[1,   100] loss: 0.472
[1,   200] loss: 0.402
[1,   300] loss: 0.301
[1,   400] loss: 0.214
[1,   500] loss: 0.276
[1,   600] loss: 0.280
[1,   700] loss: 0.246
[1,   800] loss: 0.268
[1,   900] loss: 0.346
[1,  1000] loss: 0.298
[1,  1100] loss: 0.270
[1,  1200] loss: 0.360
[1,  1300] loss: 0.210
[1,  1400] loss: 0.298
[1,  1500] loss: 0.209
[1,  1600] loss: 0.268
[1,  1700] loss: 0.192
[1,  1800] loss: 0.229
[1,  1900] loss: 0.197
[1,  2000] loss: 0.274
[1,  2100] loss: 0.303
[1,  2200] loss: 0.217
[1,  2300] loss: 0.271
[1,  2400] loss: 0.313
[1,  2500] loss: 0.162
[1,  2600] loss: 0.124
[1,  2700] loss: 0.213
[1,  2800] loss: 0.155
[1,  2900] loss: 0.267
[1,  3000] loss: 0.240
[1,  3100] loss: 0.190
[1,  3200] loss: 0.348
[1,  3300] loss: 0.270
[1,  3400] loss: 0.230
[1,  3500] loss: 0.218
[1,  3600] loss: 0.169
[1,  3700] loss: 0.202
[1,  3800] loss: 0.170
[1,  3900] loss: 0.215
[1,  4000] loss: 0.207
[1,  4100] loss: 0.198
[1,  4200] lo

Latent factors 64: Auc: 0.8951720529233058:
[1,   100] loss: 0.462
[1,   200] loss: 0.388
[1,   300] loss: 0.275
[1,   400] loss: 0.210
[1,   500] loss: 0.241
[1,   600] loss: 0.230
[1,   700] loss: 0.213
[1,   800] loss: 0.281
[1,   900] loss: 0.331
[1,  1000] loss: 0.278
[1,  1100] loss: 0.318
[1,  1200] loss: 0.346
[1,  1300] loss: 0.190
[1,  1400] loss: 0.277
[1,  1500] loss: 0.193
[1,  1600] loss: 0.211
[1,  1700] loss: 0.173
[1,  1800] loss: 0.229
[1,  1900] loss: 0.191
[1,  2000] loss: 0.271
[1,  2100] loss: 0.347
[1,  2200] loss: 0.231
[1,  2300] loss: 0.327
[1,  2400] loss: 0.303
[1,  2500] loss: 0.130
[1,  2600] loss: 0.145
[1,  2700] loss: 0.238
[1,  2800] loss: 0.173
[1,  2900] loss: 0.206
[1,  3000] loss: 0.282
[1,  3100] loss: 0.180
[1,  3200] loss: 0.301
[1,  3300] loss: 0.245
[1,  3400] loss: 0.211
[1,  3500] loss: 0.154
[1,  3600] loss: 0.194
[1,  3700] loss: 0.219
[1,  3800] loss: 0.224
[1,  3900] loss: 0.198
[1,  4000] loss: 0.271
[1,  4100] loss: 0.173
[1,  4200] lo

Latent factors 128: Auc: 0.89860491001835:


In [207]:
aucs

[0.9005884623283035,
 0.9000449631079535,
 0.8960811226825817,
 0.8951720529233058,
 0.89860491001835]